In [1]:
!pip install torch 
!pip install peft
!pip install bitsandbytes
!pip install transformers
!pip install trl 
!pip install accelerate
!pip install einops

In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
from langchain.document_loaders import HuggingFaceDatasetLoader

In [3]:
base_model = "microsoft/Phi-3-mini-128k-instruct"
#base_model = "microsoft/phi-2"
new_model = "phi-3-medicine"



# Dataset
dataset = load_dataset("wenlianghuang/finetunetest",split="train")

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right"

Generating train split:   0%|          | 0/600 [00:00<?, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base moodel
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map={"": 0}
)

model.config.use_cache = False
model.config.pretraining_tp = 1

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# LoRA configuration
peft_config = LoraConfig(
    r= 64,          
    lora_alpha= 16,
    lora_dropout=0.05, #0.1
    bias="none",
    task_type="CAUSAL_LM",
    #target_modules= ["all-linear"] 
    target_modules="all-linear",
)

In [6]:
# Set training arguments
training_arguments = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 1,
    fp16 = False,
    bf16 = False,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps = 1,
    gradient_checkpointing = True,
    max_grad_norm = 0.3,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    optim = "paged_adamw_32bit",
    lr_scheduler_type = "cosine",
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    save_steps = 0,
    logging_steps = 25,
    
)

In [7]:
# SFTTraining Ref: https://www.superannotate.com/blog/llm-fine-tuning
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    #dataset_text_field="response",
    #dataset_text_field="instruction",
    max_seq_length= None,
    tokenizer=tokenizer,
    args=training_arguments,
)

# Train model
trainer.train()

c:\Users\acer alan\AppData\Local\Programs\Python\Python311\Lib\site-packages\trl\trainer\sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/600 [00:00<?, ? examples/s]

  0%|          | 0/150 [00:00<?, ?it/s]

c:\Users\acer alan\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
You are not running the flash-attention implementation, expect numerical differences.


{'loss': 1.2114, 'grad_norm': 0.117246612906456, 'learning_rate': 0.00019075754196709572, 'epoch': 0.17}
{'loss': 0.9721, 'grad_norm': 0.10380066186189651, 'learning_rate': 0.00015611870653623825, 'epoch': 0.33}
{'loss': 0.8794, 'grad_norm': 0.10538771748542786, 'learning_rate': 0.00010541389085854176, 'epoch': 0.5}
{'loss': 0.8402, 'grad_norm': 0.11583378165960312, 'learning_rate': 5.3159155930021e-05, 'epoch': 0.67}
{'loss': 0.8552, 'grad_norm': 0.12294873595237732, 'learning_rate': 1.4314282383241096e-05, 'epoch': 0.83}
{'loss': 0.8228, 'grad_norm': 0.11900768429040909, 'learning_rate': 0.0, 'epoch': 1.0}
{'train_runtime': 1255.9634, 'train_samples_per_second': 0.478, 'train_steps_per_second': 0.119, 'train_loss': 0.9301844533284506, 'epoch': 1.0}


TrainOutput(global_step=150, training_loss=0.9301844533284506, metrics={'train_runtime': 1255.9634, 'train_samples_per_second': 0.478, 'train_steps_per_second': 0.119, 'total_flos': 1.090596671225856e+16, 'train_loss': 0.9301844533284506, 'epoch': 1.0})

In [8]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [9]:
prompt = 'When is Tsai Ing-wen resign her term of president?'
instruction = f"### Instruction: {prompt} "
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(instruction)
print(result[0]['generated_text'][len(instruction):])

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
c:\Users\acer alan\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


KeyboardInterrupt: 